In [5]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytorch_tabnet

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytorch_tabnet-4.1.0-py3-none-any.whl.metadata (15 kB)
  Using cached torch-2.3.0-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.12.0-py3-none-win_amd64.whl.metadata (1.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached pytorch_tabnet-4.1.0-py3-none-any.whl (44 kB)
Using cached torch-2.3.0-cp312-cp312-win_amd64.whl (159.7 MB)
   ---------------------------------------- 0.0/228.5 MB ? eta -:--:--
   ---------------------------------------- 0.

## 0. 필요한 라이브러리 및 모듈

In [7]:
# 판다스 넘파이
import pandas as pd
import numpy as np

# 머신러닝, 딥러닝 관련 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,accuracy_score, precision_score,roc_auc_score,f1_score,confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, LassoCV, Lasso
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier

# 리샘플링 관련
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

# 윈저라이징
from scipy.stats import mstats

# 성능평가
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

# 통계
from scipy.stats import shapiro, levene, ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm


# 경고 메세지 숨기기
import warnings 
warnings.filterwarnings('ignore')

# 시각자료(그래프) 관련 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 카운터함수
from collections import Counter

# 한글폰트 설정
# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우의 경우
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac의 경우
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 설정

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "C:\Users\SAMSUNG\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\lib\shm.dll" or one of its dependencies.

##

In [294]:
# 파일 불러오기
df = pd.read_csv("C:/Users/SAMSUNG/Documents/vscode/2024/ML/프로젝트2/data/전처리완료.csv", index_col=0)
df.reset_index(drop=True)
df.head(3)

,회사명,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,...,실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
0,(주)ES큐브,50120,2012,-16.575097,0.0,92.510776,1.564118,4.854248,-0.42,28.015517,...,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008,0.0
1,(주)ES큐브,50120,2013,1.849357,0.0,-4481.062648,6.304012,4.692085,-57.63,24.759975,...,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135,0.0
2,(주)ES큐브,50120,2014,20.099174,0.0,96.260255,17.123515,-24.126991,-73.00,21.419545,...,3.6,353600472,2.13,1099.3,-5.961831,-8.166371,2.098,101.12,-0.531182,0.0


In [295]:
# 전체 행, 전체 열을 보여주는 코드
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. 기초통계량

### 1.0 시각화

In [296]:
# 박스플랏

class Plot:
    
    def __init__(self, data, i=0):
        self.data = data
        self.i = i
        self.plot = [sns.boxplot, sns.violinplot, sns.histplot]
    def make_plot(self):
        # 서브플롯 생성
        num_plots = len(self.data.columns)
        num_cols = 6
        num_rows = (num_plots + num_cols - 1) // num_cols  # num_plots를 num_cols로 나눈 후 올림
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 3))

        # 각 플롯에 해당플롯 그리기
        for i, col in enumerate(self.data.columns):
            ax = axes[i // num_cols, i % num_cols]
            self.plot[self.i](self.data[col], ax=ax)
            ax.set_title(col, rotation=0)  # 세로로 제목을 회전

        # 빈 서브플롯 제거
        for j in range(i + 1, num_rows * num_cols):
            fig.delaxes(axes[j // num_cols, j % num_cols])

        # 서브플롯 간격 조정
        plt.subplots_adjust(hspace=0.1)  # 서브플롯 간의 세로 간격 설정

        plt.tight_layout()
        plt.show()
    def make_scatter(self):
        # scatter plot 그리기
        sns.pairplot(self.data)


### 1.1 전체 데이터 통계

In [297]:
df.describe()

,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,수지비율(관계기업투자손익 제외),사내유보 대 자기자본비율,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률,운전자본회전률,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,1주당순이익,1주당 CASH FLOW,유보율,기계투자효율,부가가치율,노동소득분배율,자기자본구성비율,유동비율,당좌비율,현금비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,부채비율,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,1주당매출액,1주당순자산,총자산영업이익률,총자산순이익률,CP전년대비증가율,경기선행지수,경기동행지수,GDP대비국가부채비율,CPI,CPI증감률,실질GDP성장률,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
count,12322.000000,12322.000000,12322.000000,12322.000000,1.232200e+04,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,1.232200e+04,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,1.232200e+04,12322.000000,12322.000000,1.232200e+04,12322.000000,12322.000000,1.232200e+04,1.232200e+04,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,1.232200e+04,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000,12322.000000
mean,80404.820646,2017.354650,11.589892,44.992473,-2.613602e+02,15.253313,21.965121,4.130053,27.146983,-33.278453,-37.278719,8.342724,-5.271419,-0.921126,98.530237,25.691118,0.926820,1.848897,2.968693,1.991158,2.780760,282.334696,9.626017,1.189982,0.416517,191.781117,107.442127,28.593710,1144.835563,20.092913,199.201686,1967.980731,4153.679420,2473.988759,1.070204e+07,-1.789411,65.080963,61.048525,293.450188,240.654464,69.118602,28.773385,8.001685e+03,100.325826,24.059593,2.612366e+03,5.695950,-24.044506,5.939279e+04,3.051818e+04,4.747940,2.692317,9.099938,93.606038,96.658456,38.663456,99.789742,-0.524319,2.520180,-0.012126,3.455235,3.821377e+08,2.001404,1139.348507,1.447890,0.990646,2.088634,102.235432,1.475844,0.082535
std,85432.961262,3.207412,81.053482,626.342832,2.767912e+04,168.510781,514.361420,53.059921,39.383127,3201.433061,3289.176149,18.283547,912.223555,129.909261,75.868045,585.176214,0.523321,1.715466,2.130156,1.211382,2.006161,5556.093983,38.787778,0.603952,11.711490,698.063365,321.596207,113.958363,8093.864804,18.101232,2107.145396,10276.558346,16524.305896,6623.618028,2.494861e+08,2173.834666,1185.249693,20.126953,395.597066,375.827631,130.675402,2617.197461,6.056366e+05,626.205949,45.027334,1.111401e+05,7.361617,2709.778130,1.747724e+05,1.199941e+05,8.850246,16.100315,56.016088,11.088988,8.388281,5.900574,6.537556,10.775520,1.337583,0.146239,0.316357,3.632233e+07,0.914931,65.861770,8.092657,11.114333,0.783651,5.425575,3.629986,0.275190
min,20.000000,2012.000000,-99.701756,-14737.300000,-3.043592e+06,-1229.876339,-99.707647,-99.710000,-3097.372431,-350051.850000,-353205.520000,-658.510000,-101015.660000,-14234.030000,-90.770000,-63839.140000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13800.730000,0.000000,0.000000,0.000000,-533.540000,-145764.420000,-128166.342333,-222830.470000,-2347.490000,-7.766250e+08,-231170.940000,-48800.259067,-101.660000,4.560000,1.810000,0.000000,-101937.619265,0.000000e+00,0.000000,-1196.020000,-4.806702e+04,-255.410000,-297559.530000,0.000000e+00,-9.298260e+03,-373.060473,-622.358125,-28.461538,75.300000,82.100000,30.800000,90.000000,-13.461538,-0.718111,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032,0.000000
25%,14440.000000,2015.000000,0.000000,-15.692500,-2.983785e+01,0.000000,-1.503288,-3.140000,12.382878,2.150000,0.912500,3.210000,1.642500,-0.370000,89.050000,9.250000,0.580000,0.860

#### 1.1.1 box plot

#### 1.1.2 violin plot



### 1.2 정상 기업 통계

#### 1.2.1 정상기업 describe

In [298]:
# 정상기업 describe
df_0 = df[df['부실'] == 0]
df_0.describe()

,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,수지비율(관계기업투자손익 제외),사내유보 대 자기자본비율,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률,운전자본회전률,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,1주당순이익,1주당 CASH FLOW,유보율,기계투자효율,부가가치율,노동소득분배율,자기자본구성비율,유동비율,당좌비율,현금비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,부채비율,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,1주당매출액,1주당순자산,총자산영업이익률,총자산순이익률,CP전년대비증가율,경기선행지수,경기동행지수,GDP대비국가부채비율,CPI,CPI증감률,실질GDP성장률,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
count,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,1.130500e+04,1.130500e+04,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.0
mean,79309.827510,2017.470234,11.692257,49.810441,9.859021,14.108263,21.399870,4.608617,27.927392,6.985081,7.084741,10.623025,7.192619,0.103785,92.785947,34.697415,0.951588,1.853942,3.106369,2.037338,2.841198,281.430591,9.733570,1.230602,0.438229,212.334081,104.647123,28.440115,1166.999564,21.745035,243.764533,2199.927323,4594.140843,2636.879211,1.159027e+07,27.408005,58.427808,62.299304,291.310864,236.846525,67.418990,18.935588,8.663949e+03,91.003099,28.154792,2.863100e+03,6.643759,9.192796,6.316726e+04,3.242202e+04,5.956893,4.462393,9.586863,94.003883,96.972260,38.839062,99.776444,-0.548381,2.515616,-0.014378,3.461097,3.834580e+08,1.985567,1140.998293,1.564250,1.085674,2.075542,102.233697,1.509815,0.0
std,84742.403827,3.155504,81.871556,644.991575,3911.481830,110.308939,519.742746,53.527232,27.255450,55.284071,104.975950,11.995499,16.614542,6.432940,14.336223,602.944593,0.520579,1.649871,2.136854,1.210661,2.014989,5647.252724,40.292164,0.598395,12.225549,708.414214,322.813708,115.801315,8279.584011,14.662017,968.220832,10468.639061,16954.541273,6878.377522,2.600878e+08,42.675787,283.093886,19.125484,324.463468,300.186066,104.978579,2659.560901,6.322888e+05,626.999178,38.763901,1.160274e+05,5.891494,34.502531,1.803140e+05,1.243138e+05,6.341068,9.863683,56.755719,10.915478,8.232343,5.870932,6.601094,10.882310,1.351619,0.147265,0.316542,3.561483e+07,0.919114,65.898147,8.164576,11.226022,0.786944,5.497874,3.668813,0.0
min,20.000000,2012.000000,-99.701756,-14737.300000,-339519.403249,-636.183557,-99.686397,-99.710000,-207.544182,-5737.150000,-4479.640000,-185.200000,-667.320000,-639.490000,-90.770000,-63839.140000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13800.730000,0.000000,0.000000,0.000000,-291.530000,-13680.040000,-128166.342333,-157950.870000,-99.690000,-2.855585e+08,-2993.710000,-20144.632768,0.150000,11.620000,8.730000,0.010000,-101937.619265,0.000000e+00,1.490000,-221.470000,-4.806702e+04,-49.560000,-863.050000,0.000000e+00,-9.298260e+03,-92.318577,-324.465151,-28.461538,75.300000,82.100000,30.800000,90.000000,-13.461538,-0.718111,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032,0.0
25%,13890.000000,2015.000000,0.000000,-17.990000,-27.925476,0.000000,-0.705825,-2.630000,12.871286,2.880000,1.650000,4.400000,2.780000,-0.400000,88.460000,12.190000,0.600000,0.880000,1.680000,1.250000,1.550000,4.718697,4.770000,0.850000,0.000000,83.23

#### 1.2.2 정상기업 box plot

### 1.3 부실 기업 통계

In [299]:
# 정상기업 describe
df_0 = df[df['부실'] == 0]
df_0.describe()

,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,수지비율(관계기업투자손익 제외),사내유보 대 자기자본비율,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률,운전자본회전률,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,1주당순이익,1주당 CASH FLOW,유보율,기계투자효율,부가가치율,노동소득분배율,자기자본구성비율,유동비율,당좌비율,현금비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,부채비율,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,1주당매출액,1주당순자산,총자산영업이익률,총자산순이익률,CP전년대비증가율,경기선행지수,경기동행지수,GDP대비국가부채비율,CPI,CPI증감률,실질GDP성장률,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
count,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,1.130500e+04,1.130500e+04,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,1.130500e+04,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.000000,11305.0
mean,79309.827510,2017.470234,11.692257,49.810441,9.859021,14.108263,21.399870,4.608617,27.927392,6.985081,7.084741,10.623025,7.192619,0.103785,92.785947,34.697415,0.951588,1.853942,3.106369,2.037338,2.841198,281.430591,9.733570,1.230602,0.438229,212.334081,104.647123,28.440115,1166.999564,21.745035,243.764533,2199.927323,4594.140843,2636.879211,1.159027e+07,27.408005,58.427808,62.299304,291.310864,236.846525,67.418990,18.935588,8.663949e+03,91.003099,28.154792,2.863100e+03,6.643759,9.192796,6.316726e+04,3.242202e+04,5.956893,4.462393,9.586863,94.003883,96.972260,38.839062,99.776444,-0.548381,2.515616,-0.014378,3.461097,3.834580e+08,1.985567,1140.998293,1.564250,1.085674,2.075542,102.233697,1.509815,0.0
std,84742.403827,3.155504,81.871556,644.991575,3911.481830,110.308939,519.742746,53.527232,27.255450,55.284071,104.975950,11.995499,16.614542,6.432940,14.336223,602.944593,0.520579,1.649871,2.136854,1.210661,2.014989,5647.252724,40.292164,0.598395,12.225549,708.414214,322.813708,115.801315,8279.584011,14.662017,968.220832,10468.639061,16954.541273,6878.377522,2.600878e+08,42.675787,283.093886,19.125484,324.463468,300.186066,104.978579,2659.560901,6.322888e+05,626.999178,38.763901,1.160274e+05,5.891494,34.502531,1.803140e+05,1.243138e+05,6.341068,9.863683,56.755719,10.915478,8.232343,5.870932,6.601094,10.882310,1.351619,0.147265,0.316542,3.561483e+07,0.919114,65.898147,8.164576,11.226022,0.786944,5.497874,3.668813,0.0
min,20.000000,2012.000000,-99.701756,-14737.300000,-339519.403249,-636.183557,-99.686397,-99.710000,-207.544182,-5737.150000,-4479.640000,-185.200000,-667.320000,-639.490000,-90.770000,-63839.140000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13800.730000,0.000000,0.000000,0.000000,-291.530000,-13680.040000,-128166.342333,-157950.870000,-99.690000,-2.855585e+08,-2993.710000,-20144.632768,0.150000,11.620000,8.730000,0.010000,-101937.619265,0.000000e+00,1.490000,-221.470000,-4.806702e+04,-49.560000,-863.050000,0.000000e+00,-9.298260e+03,-92.318577,-324.465151,-28.461538,75.300000,82.100000,30.800000,90.000000,-13.461538,-0.718111,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032,0.0
25%,13890.000000,2015.000000,0.000000,-17.990000,-27.925476,0.000000,-0.705825,-2.630000,12.871286,2.880000,1.650000,4.400000,2.780000,-0.400000,88.460000,12.190000,0.600000,0.880000,1.680000,1.250000,1.550000,4.718697,4.770000,0.850000,0.000000,83.23

### 1.4 정상 부실 비교

##

In [300]:
#  데이터프레임의 일부만 보여주는 코드
pd.options.display.max_rows = 60
pd.options.display.max_columns = 20


## 2. Train/Test Split

### 2.1 Train/Test Split

In [301]:
# X_data와 y_data 분리
X_data = df.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
y_data = df['부실']

In [302]:
# Train/Test Split
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=0.3,stratify=y_data,random_state = 42)

#### 2.1.1 Train set과 Test set 차이

### 2.2 Train 통계량

In [303]:
X_train.describe()

,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,...,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률
count,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,...,8625.000000,8625.000000,8.625000e+03,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000,8625.000000
mean,12.179259,40.875086,-26.837197,16.418025,25.548499,4.232736,27.475535,-0.171474,-0.528053,8.333055,...,-0.013512,3.451455,3.817053e+08,2.018444,1139.019861,1.445508,1.021900,2.101739,102.293459,1.484610
std,91.516726,521.579154,4425.460247,192.408894,605.247903,57.056542,27.629897,141.437502,170.770375,18.981494,...,0.147356,0.317217,3.638170e+07,0.920778,66.230775,8.094393,11.120991,0.789408,5.451577,3.632635
min,-99.701756,-14737.300000,-339519.403249,-1229.876339,-99.686397,-99.710000,-207.544182,-7997.200000,-7978.710000,-658.510000,...,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032
25%,0.000000,-15.360000,-29.560253,0.000000,-1.377341,-3.140000,12.365008,2.170000,0.910000,3.220000,...,-0.020000,3.200000,3.585138e+08,1.520000,1070.600000,-5.204285,-8.166371,1.638000,100.000000,-0.531182
50%,2.938018,0.000000,0.000000,3.908842,1.643866,1.120000,20.345081,5.420000,4.280000,8.440000,...,0.020000,3.500000,3.933325e+08,1.670000,1115.700000,-3.297016,-0.724218,1.817000,100.460000,0.029872
75%,12.259146,26.210000,50.278705,12.390279,14.037457,7.500000,36.492825,10.580000,9.160000,15.030000,...,0.110000,3.600000,3.990431e+08,2.660000,1188.800000,6.142843,6.135630,2.820000,103.310000,3.441763
max,6181.288402,24820.190000,68410.347509,12465.900000,40693.187936,3478.130000,100.000000,82.100000,8332.240000,169.210000,...,0.180000,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635


#### 2.3.1 부실기업과 정상기업의 describe

In [304]:
Train = X_train.copy()
Train['부실'] = y_train
Train

,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,...,실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
5770,0.000000,-29.33,0.000000,0.000000,0.000000,-8.46,53.860502,9.52,9.31,3.63,...,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135,0.0
675,0.000000,-69.19,0.000000,0.000000,0.000000,-2.08,8.455136,5.05,4.45,3.14,...,3.4,361701404,1.52,1207.7,-3.297016,-4.409925,1.638,95.30,-1.813311,0.0
729,10.593594,-10.48,-17.587225,3.238363,13.705315,1.29,26.593903,8.81,4.87,15.43,...,3.4,361701404,1.52,1207.7,-3.297016,-4.409925,1.638,95.30,-1.813311,0.0
8846,9.493539,0.00,1981.751912,33.970142,145.029092,-20.67,15.833139,6.18,4.89,30.20,...,3.4,438319248,1.29,1188.8,14.290000,14.987673,1.798,106.38,6.380000,0.0
5307,3.047819,-49.64,-12.414234,3.983928,-30.176684,2.70,47.614967,20.97,17.37,10.67,...,3.6,393332482,1.93,1115.7,0.747646,5.893729,1.817,100.43,1.876648,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,1.091685,-28.63,42.855980,0.353277,16.447390,-4.44,29.861229,3.85,5.87,2.28,...,3.0,399043064,3.98,1264.5,16.641876,20.573937,3.722,115.29,8.375635,0.0
1340,12.926085,-2.58,-29.555928,9.581278,2.816797,3.35,36.352206,19.58,11.57,16.82,...,3.6,353600472,2.13,1099.3,-5.961831,-8.166371,2.098,101.12,-0.531182,0.0
123,3.717672,-17.15,-13.787787,10.092124,-0.803211,10.77,36.675463,10.88,9.28,16.72,...,4.2,430117234,0.66,1086.3,-5.204285,-9.500000,0.976,100.00,-0.457894,0.0
1073,14.998660,65.45,35.780740,11.371989,24.486564,5.99,29.503162,20.45,16.39,16.36,...,3.0,399043064,3.98,1264.5,16.641876,20.573937,3.722,115.29,8.375635,0.0


In [305]:
Train[Train['부실']==0].describe()

,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,...,실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
count,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,...,7913.000000,7.913000e+03,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.000000,7913.0
mean,12.109658,44.728740,-6.826539,14.159694,25.548368,4.704865,28.015319,7.527701,7.757840,10.642979,...,3.457121,3.829701e+08,2.003300,1140.548932,1.550830,1.099459,2.089128,102.292472,1.513579,0.0
std,92.626770,539.096823,4584.450050,117.219402,619.205609,57.429860,27.396598,11.997956,109.131215,11.682026,...,0.317492,3.572050e+07,0.924964,66.260597,8.162947,11.234757,0.792934,5.517461,3.670824,0.0
min,-99.701756,-14737.300000,-339519.403249,-636.183557,-99.686397,-99.710000,-207.544182,-507.950000,-529.580000,-121.700000,...,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032,0.0
25%,0.000000,-17.780000,-27.645570,0.000000,-0.623929,-2.610000,12.902723,2.920000,1.670000,4.410000,...,3.200000,3.585138e+08,1.520000,1086.300000,-5.204285,-8.166371,1.638000,100.000000,-0.531182,0.0
50%,3.306637,0.000000,0.000000,4.410744,2.226599,1.380000,20.713798,5.970000,4.790000,9.240000,...,3.500000,3.933325e+08,1.670000,1115.700000,-3.297016,0.831050,1.817000,100.460000,0.029872,0.0
75%,12.437541,30.490000,51.809049,12.598444,14.159054,7.610000,37.070836,11.170000,9.780000,15.690000,...,3.600000,3.990431e+08,2.660000,1188.800000,6.142843,6.135630,2.820000,103.310000,3.441763,0.0
max,6181.288402,24820.190000,68410.347509,8527.244514,40693.187936,3478.130000,100.000000,82.100000,8332.240000,169.210000,...,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635,0.0


In [306]:
Train[Train['부실']==1].describe()

,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,...,실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,...,712.000000,7.120000e+02,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.0
mean,12.952791,-1.953511,-249.230925,41.516580,25.549948,-1.014396,21.476505,-85.738287,-92.615520,-17.338890,...,3.388483,3.676478e+08,2.186756,1122.026124,0.274984,0.159930,2.241906,102.304424,1.162660,1.0
std,78.177961,252.311120,1901.997446,543.556880,420.305442,52.482058,29.462862,482.752233,460.366673,46.175678,...,0.307400,4.050072e+07,0.855645,63.509863,7.191219,9.733300,0.735193,4.660404,3.161926,0.0
min,-96.705952,-4532.630000,-34494.219109,-1229.876339,-99.505886,-98.940000,-97.919116,-7997.200000,-7978.710000,-658.510000,...,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032,1.0
25%,-10.326677,0.000000,-90.534308,-15.852684,-12.697612,-13.300000,6.563161,-25.382500,-43.420000,-21.737500,...,3.100000,3.309600e+08,1.530000,1070.600000,-4.268391,-7.917155,1.662000,100.000000,-0.531182,1.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,-1.665000,14.798043,-7.315000,-13.005000,-7.245000,...,3.400000,3.617014e+08,2.130000,1099.300000,-2.367798,-0.724218,2.098000,101.120000,0.692008,1.0
75%,8.559703,0.000000,34.187856,5.513521,10.262000,4.737500,29.683225,0.000000,-2.170000,0.000000,...,3.500000,3.978761e+08,2.890000,1172.500000,0.747646,5.893729,2.820000,103.310000,1.876648,1.0
max,834.037855,2669.910000,6463.126180,12465.900000,10907.900318,1154.550000,100.000000,21.970000,2096.990000,39.680000,...,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635,1.0


#### 2.3.2. 부실기업과 정상기업의 히스토그램

## 3. winsorizing

In [307]:
def winsorize_series(series, lower_bound, upper_bound):
    return np.clip(series, lower_bound, upper_bound)

def apply_winsorizing(train_df, test_df, lower_quantile=0.005, upper_quantile=0.995):
    train_winsorized = train_df.copy()
    test_winsorized = test_df.copy()
    
    for col in train_df.columns:
        lower_bound = train_df[col].quantile(lower_quantile)
        upper_bound = train_df[col].quantile(upper_quantile)
        
        train_winsorized[col] = winsorize_series(train_df[col], lower_bound, upper_bound)
        test_winsorized[col] = winsorize_series(test_df[col], lower_bound, upper_bound)
    
    return train_winsorized, test_winsorized

# 트레이닝 및 테스트 데이터셋에 윈저라이징 적용
X_train, X_test = apply_winsorizing(X_train, X_test)


## 4. Scailing

In [308]:
class Scailing:
    def __init__(self, Train_data, Test_data):
        self.Train_data = Train_data
        self.Test_data = Test_data

    def Rbs(self):
        Rbscaler = RobustScaler()
        Rbscaler.fit(self.Train_data)
        X_scaled_train = Rbscaler.transform(self.Train_data)
        X_scaled_test = Rbscaler.transform(self.Test_data)
        return X_scaled_train, X_scaled_test

    def Stds(self):
        Stdscaler = StandardScaler()
        Stdscaler.fit(self.Train_data)
        X_scaled_train = Stdscaler.transform(self.Train_data)
        X_scaled_test = Stdscaler.transform(self.Test_data)
        return X_scaled_train, X_scaled_test


    def Mms(self):
        Mmscaler = MinMaxScaler()
        Mmscaler.fit(self.Train_data)
        X_scaled_train = Mmscaler.transform(self.Train_data)
        X_scaled_test = Mmscaler.transform(self.Test_data)
        return X_scaled_train, X_scaled_test

## 5. Resampling

In [309]:
class Resampling:
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def bds(self):
        X_resampled_train, y_resampled_train = BorderlineSMOTE(random_state=0).fit_resample(self.X_train, self.y_train)    
        return X_resampled_train, y_resampled_train

    def tomek(self):
        smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
        X_resampled_train, y_resampled_train = smoteto.fit_resample(self.X_train, self.y_train)    
        return X_resampled_train, y_resampled_train

    def adasyn(self):
        X_resampled_train, y_resampled_train = ADASYN(random_state=42).fit_resample(self.X_train, self.y_train)    
        return X_resampled_train, y_resampled_train    


## 6. Feature Selection

### 6.1 라쏘

In [320]:
Las = LassoCV(random_state=0).fit(X_train, y_train)
best_alpha = Las.alpha_
X_corr = pd.DataFrame(X_train).corr().values
print(best_alpha)
print(Las.coef_)


1160.4557002161357
[ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -2.65715318e-10 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -9.32593894e-08 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -8.79941398e-10
  0.00

In [321]:
lasso_cv = LassoCV(cv=5, random_state=0)
lasso_cv.fit(X_train, y_train)

# 최적의 알파 값 출력
best_alpha = lasso_cv.alpha_
print("Best alpha:", best_alpha)

# 각 알파 값에 대한 R^2 점수 출력
alphas = lasso_cv.alphas_
r2_scores = lasso_cv.mse_path_.mean(axis=1)
print("Alphas and corresponding R^2 scores:")
for alpha, r2_score in zip(alphas, r2_scores):
    print(f"Alpha: {alpha:.5f}, R^2: {r2_score:.5f}")

# 최적의 알파 값으로 모델 학습 및 평가
lasso_best = LassoCV(alphas=[best_alpha])
lasso_best.fit(X_train, y_train)
print("Model R^2 score on test data:", lasso_best.score(X_test, y_test))

Best alpha: 1160.4557002161357
Alphas and corresponding R^2 scores:
Alpha: 1160455.70022, R^2: 0.07574
Alpha: 1082244.86993, R^2: 0.07566
Alpha: 1009305.18785, R^2: 0.07555
Alpha: 941281.39621, R^2: 0.07545
Alpha: 877842.18047, R^2: 0.07536
Alpha: 818678.55554, R^2: 0.07528
Alpha: 763502.36092, R^2: 0.07522
Alpha: 712044.85714, R^2: 0.07516
Alpha: 664055.41690, R^2: 0.07511
Alpha: 619300.30432, R^2: 0.07506
Alpha: 577561.53653, R^2: 0.07503
Alpha: 538635.82200, R^2: 0.07499
Alpha: 502333.57035, R^2: 0.07496
Alpha: 468477.96896, R^2: 0.07494
Alpha: 436904.12179, R^2: 0.07492
Alpha: 407458.24625, R^2: 0.07490
Alpha: 379996.92417, R^2: 0.07488
Alpha: 354386.40329, R^2: 0.07487
Alpha: 330501.94579, R^2: 0.07486
Alpha: 308227.22080, R^2: 0.07484
Alpha: 287453.73772, R^2: 0.07484
Alpha: 268080.31787, R^2: 0.07483
Alpha: 250012.60168, R^2: 0.07482
Alpha: 233162.58909, R^2: 0.07481
Alpha: 217448.21095, R^2: 0.07481
Alpha: 202792.92931, R^2: 0.07480
Alpha: 189125.36460, R^2: 0.07480
Alpha: 1763

In [311]:
def feature_selection_lasso(X, y, alpha=best_alpha):
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X, y)
    selected_features = X.columns[lasso.coef_ != 0]
    return selected_features

# 라쏘를 사용하여 피처 선택
selected_features = feature_selection_lasso(X_train, y_train)

print("Selected features:", selected_features)

Selected features: Index(['기계투자효율', '1주당매출액', '외환보유액(천달러)'], dtype='object')


In [322]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 알파 값을 0과 1 사이로 설정
alphas = np.linspace(0.01, 1, 100)

# LassoCV를 사용하여 최적의 알파 값 찾기
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=0)
lasso_cv.fit(X_train, y_train)

# 최적의 알파 값 출력
best_alpha = lasso_cv.alpha_
print("Best alpha:", best_alpha)

# 각 알파 값에 대한 평균 교차 검증 MSE 출력
mse_path = lasso_cv.mse_path_.mean(axis=1)
print("Alphas and corresponding average MSE scores:")
for alpha, mse in zip(alphas, mse_path):
    print(f"Alpha: {alpha:.5f}, Average MSE: {mse:.5f}")

# 최적의 알파 값으로 모델 평가
y_pred_train = lasso_cv.predict(X_train)
y_pred_test = lasso_cv.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print("Train R^2:", r2_train)
print("Test R^2:", r2_test)
print("Train RMSE:", rmse_train)
print("Test RMSE:", rmse_test)


Best alpha: 0.01
Alphas and corresponding average MSE scores:
Alpha: 0.01000, Average MSE: 0.05380
Alpha: 0.02000, Average MSE: 0.05376
Alpha: 0.03000, Average MSE: 0.05373
Alpha: 0.04000, Average MSE: 0.05369
Alpha: 0.05000, Average MSE: 0.05366
Alpha: 0.06000, Average MSE: 0.05362
Alpha: 0.07000, Average MSE: 0.05358
Alpha: 0.08000, Average MSE: 0.05355
Alpha: 0.09000, Average MSE: 0.05351
Alpha: 0.10000, Average MSE: 0.05348
Alpha: 0.11000, Average MSE: 0.05345
Alpha: 0.12000, Average MSE: 0.05341
Alpha: 0.13000, Average MSE: 0.05338
Alpha: 0.14000, Average MSE: 0.05335
Alpha: 0.15000, Average MSE: 0.05332
Alpha: 0.16000, Average MSE: 0.05328
Alpha: 0.17000, Average MSE: 0.05325
Alpha: 0.18000, Average MSE: 0.05321
Alpha: 0.19000, Average MSE: 0.05317
Alpha: 0.20000, Average MSE: 0.05312
Alpha: 0.21000, Average MSE: 0.05307
Alpha: 0.22000, Average MSE: 0.05301
Alpha: 0.23000, Average MSE: 0.05295
Alpha: 0.24000, Average MSE: 0.05289
Alpha: 0.25000, Average MSE: 0.05282
Alpha: 0.2600

In [323]:
def feature_selection_lasso(X, y, alpha = 0.01):
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X, y)
    selected_features = X.columns[lasso.coef_ != 0]
    return selected_features

# 라쏘를 사용하여 피처 선택
selected_features = feature_selection_lasso(X_train, y_train)

print("Selected features:", selected_features)

Selected features: Index(['총자본증가율', '정상영업이익증가율', '순이익증가율', '자기자본증가율', '매출액증가율', '종업원수증가율',
       '매출액총이익률', '매출액정상영업이익률', '매출액순이익률', '자기자본정상영업이익률', '자기자본순이익률',
       '금융비용부담률', '수지비율(관계기업투자손익 제외)', '사내유보 대 자기자본비율', '타인자본회전률', '당좌자산회전률',
       '재고자산회전률', '매출채권회전률', '종업원1인당 부가가치', '노동장비율', '기계장비율', '자본집약도',
       '총자본투자효율', '설비투자효율', '1주당순이익', '1주당 CASH FLOW', '유보율', '기계투자효율',
       '부가가치율', '노동소득분배율', '자기자본구성비율', '유동비율', '당좌비율', '현금비율',
       '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율', '부채비율', 'CASH FLOW 대 부채비율',
       'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '1주당매출액',
       '1주당순자산', '총자산순이익률', 'CP전년대비증가율', 'CPI', '외환보유액(천달러)', '환율',
       '수입물가지수증감률', '생산자물가지수'],
      dtype='object')


In [313]:
def feature_selection_decision_tree(X_train, y_train, max_features=None):
    dt = DecisionTreeClassifier(max_features=max_features, random_state=42)
    dt.fit(X_train, y_train)
    importances = dt.feature_importances_
    selected_features_indices = np.argsort(importances)[::-1]
    selected_features = X_train.columns[selected_features_indices]
    return selected_features

# Decision Tree를 사용하여 피처 선택
selected_features_dt = feature_selection_decision_tree(X_train, y_train)

print("Selected features:", selected_features_dt)

Selected features: Index(['매출액정상영업이익률', '사내유보 대 자기자본비율', 'CASH FLOW 대 매출액비율', '금융비용부담률',
       '타인자본회전률', '정상영업이익증가율', '총자본증가율', '총자산영업이익률', '외환보유액(천달러)', '순이익증가율',
       '유보율', '자기자본증가율', '1주당매출액', 'CASH FLOW 대 총자본비율', '재고자산회전률', '1주당순이익',
       'CASH FLOW 대 부채비율', '유동비율', '총자산순이익률', '자기자본구성비율', '자기자본회전률', '매출액총이익률',
       '유동자산회전률', 'CP전년대비증가율', '재고자산 대 순운전자본비율', '매출액순이익률', '기계투자효율',
       '종업원수증가율', '순운전자본회전률', '생산자물가지수증감률', '매출채권회전률', '노동소득분배율', '부채비율',
       '1주당 CASH FLOW', '당좌자산회전률', '운전자본회전률', '노동장비율', '당좌비율', '매출액증가율',
       'CPI증감률', '경기동행지수', '총자본투자효율', '기계장비율', '매출채권 대 매입채무비율', '현금비율',
       '1주당순자산', '자기자본정상영업이익률', '종업원1인당 부가가치', '자본집약도', 'CD이자율', '자기자본순이익률',
       '부가가치율', 'CPI', 'CASH FLOW 대 차입금비율', '수입물가지수증감률', '장단기금리차(전월차)(%p)',
       '수출물가지수증감률', '3년국고채수익률', '수지비율(관계기업투자손익 제외)', '환율', '설비투자효율', '실업률',
       '실질GDP성장률', 'GDP대비국가부채비율', '경기선행지수', '생산자물가지수', '총자본회전률'],
      dtype='object')


In [314]:
len(set(selected_features))

36

### 6.2 t-test

In [315]:
from scipy.stats import shapiro, levene, ttest_ind

# 정규성 확인 (Shapiro-Wilk 검정)
normality_results = {}
for column in Train[:-1].columns:
    stat_group1, p_group1 = shapiro(Train.loc[Train['부실'] == 0, column])
    stat_group2, p_group2 = shapiro(Train.loc[Train['부실'] == 1, column])
    normality_results[column] = {'Group 1': (stat_group1, p_group1), 'Group 2': (stat_group2, p_group2)}

print('Shapiro-Wilk test - Normality:')
for column, results in sorted(normality_results.items(), key=lambda x: x[1]['Group 1'][1]):
    stat_group1, p_group1 = results['Group 1']
    stat_group2, p_group2 = results['Group 2']
    print('%s: Group 1 - stat=%.3f, p-value=%.3f; Group 2 - stat=%.3f, p-value=%.3f' % (column, stat_group1, p_group1, stat_group2, p_group2))

# 등분산성 확인 (Levene's test)
variance_results = {}
for column in Train[:-1].columns:
    stat_levene, p_levene = levene(Train.loc[Train['부실'] == 0, column], Train.loc[Train['부실'] == 1, column])
    variance_results[column] = (stat_levene, p_levene)

print('Levene\'s test for Homogeneity of Variance:')
for column, (stat_levene, p_levene) in variance_results.items():
    print('%s: stat=%.3f, p-value=%.3f' % (column, stat_levene, p_levene))

# t-검정 수행
t_test_results = {}
for column in Train[:-1].columns:
    equal_var = variance_results[column][1] > 0.05  # 등분산성 가정이 성립하는 경우
    t_stat, p_ttest = ttest_ind(Train.loc[Train['부실'] == 0, column], Train.loc[Train['부실'] == 1, column], equal_var=equal_var)
    t_test_results[column] = (t_stat, p_ttest)

print('t-test results:')
for column, (t_stat, p_ttest) in sorted(t_test_results.items(), key=lambda x: x[1][1]):
    print('%s: stat=%.3f, p-value=%.3f' % (column, t_stat, p_ttest))


Shapiro-Wilk test - Normality:
총자본증가율: Group 1 - stat=0.115, p-value=0.000; Group 2 - stat=0.487, p-value=0.000
정상영업이익증가율: Group 1 - stat=0.105, p-value=0.000; Group 2 - stat=0.143, p-value=0.000
순이익증가율: Group 1 - stat=0.026, p-value=0.000; Group 2 - stat=0.156, p-value=0.000
자기자본증가율: Group 1 - stat=0.100, p-value=0.000; Group 2 - stat=0.126, p-value=0.000
매출액증가율: Group 1 - stat=0.019, p-value=0.000; Group 2 - stat=0.065, p-value=0.000
종업원수증가율: Group 1 - stat=0.129, p-value=0.000; Group 2 - stat=0.364, p-value=0.000
매출액총이익률: Group 1 - stat=0.781, p-value=0.000; Group 2 - stat=0.873, p-value=0.000
매출액정상영업이익률: Group 1 - stat=0.593, p-value=0.000; Group 2 - stat=0.158, p-value=0.000
매출액순이익률: Group 1 - stat=0.029, p-value=0.000; Group 2 - stat=0.206, p-value=0.000
자기자본정상영업이익률: Group 1 - stat=0.863, p-value=0.000; Group 2 - stat=0.433, p-value=0.000
자기자본순이익률: Group 1 - stat=0.585, p-value=0.000; Group 2 - stat=0.020, p-value=0.000
금융비용부담률: Group 1 - stat=0.394, p-value=0.000; Group 2 - stat

In [316]:
def feature_selection(X, y, threshold=0.05):
    p_values = {}
    for column in X.columns:
        group1 = X[y == 0][column]
        group2 = X[y == 1][column]
        t_stat, p_value = ttest_ind(group1, group2)
        p_values[column] = p_value
    
    selected_features = [k for k, v in sorted(p_values.items(), key=lambda item: item[1])]
    selected_features = [feature for feature in selected_features if p_values[feature] < threshold]
    
    return selected_features

# 예시: X_train과 y_train을 가지고 특성 선택 수행
selected_features = feature_selection(X_train, y_train)
print("Selected features:", selected_features)

Selected features: ['매출액정상영업이익률', '매출액순이익률', '자기자본정상영업이익률', '자기자본순이익률', '수지비율(관계기업투자손익 제외)', '사내유보 대 자기자본비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '총자산영업이익률', '총자산순이익률', '부가가치율', '총자본투자효율', '금융비용부담률', 'CASH FLOW 대 부채비율', '순운전자본회전률', '타인자본회전률', '부채비율', '자기자본구성비율', '총자본회전률', '종업원1인당 부가가치', '유동자산회전률', '유보율', '당좌자산회전률', '경기동행지수', '외환보유액(천달러)', '경기선행지수', '1주당 CASH FLOW', '순이익증가율', 'GDP대비국가부채비율', '종업원수증가율', '1주당순이익', '1주당순자산', '환율', '1주당매출액', '설비투자효율', '매출액총이익률', '실업률', 'CASH FLOW 대 차입금비율', 'CD이자율', '3년국고채수익률', '장단기금리차(전월차)(%p)', '노동장비율', '노동소득분배율', '수출물가지수증감률', '정상영업이익증가율', '생산자물가지수증감률', 'CP전년대비증가율', '재고자산 대 순운전자본비율', '수입물가지수증감률']


In [317]:
# feature_tmp = [feature for feature in selected_features if feature in selected_features_dt]

In [318]:
# feature_tmp

### 6.3 후진제거법

In [319]:
def backward_elimination(X, y, significance_level=0.05):
    X = sm.add_constant(X)  # 상수항 추가
    original_features = X.columns.tolist()
    while True:
        model = sm.OLS(y, X).fit()
        p_values = model.pvalues
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            X = X.drop(columns=[excluded_feature])
        else:
            break
    selected_features = X.columns.tolist()
    removed_features = list(set(original_features) - set(selected_features))
    return model, selected_features, removed_features

# 후진제거법 수행
model, selected_features, removed_features = backward_elimination(X_train, y_train)

print('Selected features:', selected_features)
print('Removed features:', removed_features)
print(model.summary())


Selected features: ['자기자본증가율', '매출액증가율', '종업원수증가율', '매출액정상영업이익률', '매출액순이익률', '자기자본정상영업이익률', '금융비용부담률', '수지비율(관계기업투자손익 제외)', '사내유보 대 자기자본비율', '자기자본회전률', '유동자산회전률', '매출채권회전률', '순운전자본회전률', '운전자본회전률', '자본집약도', '총자본투자효율', '1주당순이익', '노동소득분배율', '유동비율', '당좌비율', '부채비율', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '1주당순자산', '총자산영업이익률', '총자산순이익률', 'CP전년대비증가율', '경기동행지수', 'GDP대비국가부채비율', 'CPI', '외환보유액(천달러)', '수입물가지수증감률', '생산자물가지수', '생산자물가지수증감률']
Removed features: ['자기자본순이익률', 'const', '기계투자효율', 'CD이자율', '실업률', '매출액총이익률', '설비투자효율', '정상영업이익증가율', '타인자본회전률', '총자본회전률', '순이익증가율', '재고자산회전률', '당좌자산회전률', '부가가치율', '매출채권 대 매입채무비율', '자기자본구성비율', 'CPI증감률', '실질GDP성장률', '기계장비율', '노동장비율', '총자본증가율', '1주당 CASH FLOW', '환율', '종업원1인당 부가가치', '수출물가지수증감률', '재고자산 대 순운전자본비율', '1주당매출액', '3년국고채수익률', '장단기금리차(전월차)(%p)', '유보율', 'CASH FLOW 대 차입금비율', '현금비율', '경기선행지수']
                                 OLS Regression Results                                
Dep. Variable:                     부실   R-squared (uncentered

In [281]:
def calculate_vif(X):
    # X = add_constant(X)
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_result = calculate_vif(X_train)
vif_result

,Feature,VIF
0,CASH FLOW 대 총자본비율,4.737242
1,CASH FLOW 대 부채비율,3.503434
2,총자산영업이익률,5.611863
3,총자산순이익률,4.278556
4,총자본투자효율,7.017372
5,부가가치율,4.398880
6,자기자본구성비율,3.888224
7,부채비율,1.698881


## 7. Modeling(Hyperparameter Tunning) 

In [282]:
df = pd.DataFrame()

In [283]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred_train = logit.predict(X_train)
y_pred_test = logit.predict(X_test)

log_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
log_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['logit_train'] = log_train
df['logit_test'] = log_test

print(confusion_matrix(y_test,y_pred_test))

[[3345   47]
 [ 159  146]]


In [284]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

rf_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
rf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['rf_train'] = rf_train
df['rf_test'] = rf_test


print(confusion_matrix(y_test,y_pred_test))

[[3329   63]
 [ 138  167]]


In [285]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred_train = xgb.predict(X_train)
y_pred_test = xgb.predict(X_test)

xgb_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
xgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['xgb_train'] = xgb_train
df['xgb_test'] = xgb_test


print(confusion_matrix(y_test,y_pred_test))

[[3312   80]
 [ 123  182]]


In [286]:
lgb = LGBMClassifier(random_state=42)
lgb.fit(X_train, y_train)
y_pred_train = lgb.predict(X_train)
y_pred_test = lgb.predict(X_test)

lgb_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
lgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['lgb_train'] = lgb_train
df['lgb_test'] = lgb_test


print(confusion_matrix(y_test,y_pred_test))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 712, number of negative: 7913
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 8625, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082551 -> initscore=-2.408184
[LightGBM] [Info] Start training from score -2.408184
[[3314   78]
 [ 126  179]]


In [287]:
cat = CatBoostClassifier(random_state=42, verbose=0)
cat.fit(X_train, y_train)
y_pred_train = cat.predict(X_train)
y_pred_test = cat.predict(X_test)

cat_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
cat_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['cat_train'] = cat_train
df['cat_test'] = cat_test


print(confusion_matrix(y_test,y_pred_test))

[[3322   70]
 [ 130  175]]


In [288]:
svc_clf = SVC() 
svc_clf.fit(X_train,y_train)
y_pred_train = svc_clf.predict(X_train)
y_pred_test = svc_clf.predict(X_test)

svc_clf_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
svc_clf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['svm_train'] = svc_clf_train
df['svm_test'] = svc_clf_test


print(confusion_matrix(y_test,y_pred_test))

[[3366   26]
 [ 191  114]]


In [289]:
clf = TabNetClassifier()

# 모델 훈련
clf.fit(
    X_train.values, y_train.values,
    eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],
    eval_name=['train', 'test'],
    eval_metric=['accuracy'],
    max_epochs=10,
    patience=10,
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

epoch 0  | loss: 0.29479 | train_accuracy: 0.92359 | test_accuracy: 0.9267  |  0:00:02s
epoch 1  | loss: 0.17701 | train_accuracy: 0.93078 | test_accuracy: 0.93075 |  0:00:07s
epoch 2  | loss: 0.1571  | train_accuracy: 0.93252 | test_accuracy: 0.93319 |  0:00:10s
epoch 3  | loss: 0.15507 | train_accuracy: 0.93786 | test_accuracy: 0.93698 |  0:00:13s
epoch 4  | loss: 0.15467 | train_accuracy: 0.93519 | test_accuracy: 0.93698 |  0:00:16s
epoch 5  | loss: 0.14796 | train_accuracy: 0.93577 | test_accuracy: 0.93806 |  0:00:19s
epoch 6  | loss: 0.14526 | train_accuracy: 0.93739 | test_accuracy: 0.93914 |  0:00:22s
epoch 7  | loss: 0.14435 | train_accuracy: 0.93925 | test_accuracy: 0.94347 |  0:00:25s
epoch 8  | loss: 0.14055 | train_accuracy: 0.94122 | test_accuracy: 0.94374 |  0:00:28s
epoch 9  | loss: 0.14094 | train_accuracy: 0.94157 | test_accuracy: 0.94266 |  0:00:31s
Stop training because you reached max_epochs = 10 with best_epoch = 8 and best_test_accuracy = 0.94374


In [290]:
y_pred_train = clf.predict(X_train.values)
y_pred_test = clf.predict(X_test.values)

tab_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
tab_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['tabnet_train'] = tab_train
df['tabnet_test'] = tab_test


print(confusion_matrix(y_test,y_pred_test))

[[3346   46]
 [ 162  143]]


In [291]:
df.index = ['accuracy','precision','recall','f1-score','roc-auc']
df

,logit_train,logit_test,rf_train,rf_test,xgb_train,xgb_test,lgb_train,lgb_test,cat_train,cat_test,svm_train,svm_test,tabnet_train,tabnet_test
accuracy,0.944928,0.944279,1.0,0.945632,0.999768,0.945091,0.992696,0.944820,0.985391,0.945902,0.940406,0.941304,0.941217,0.943738
precision,0.772414,0.756477,1.0,0.726087,1.000000,0.694656,0.968254,0.696498,0.986711,0.714286,0.823529,0.814286,0.737819,0.756614
recall,0.471910,0.478689,1.0,0.547541,0.997191,0.596721,0.942416,0.586885,0.834270,0.573770,0.353933,0.373770,0.446629,0.468852
f1-score,0.585876,0.586345,1.0,0.624299,0.998594,0.641975,0.955160,0.637011,0.904110,0.636364,0.495088,0.512360,0.556430,0.578947
roc-auc,0.729700,0.732416,1.0,0.764484,0.998596,0.786568,0.969818,0.781945,0.916629,0.776567,0.673554,0.683053,0.716174,0.727646
